In [5]:
from ipywidgets import interact, SelectMultiple, Dropdown

def select_vessels():

    #Create labels for the dropdown list:
    #('One', 1)
    vessels = [('Commercial Fishing Vessels',1),
                ('Skiffs',2),
                ('Mono-hull Sailing Vessel',3),
                ('Rustic raft with sail',4),
                ('Rustic raft without sail',5),
                ('Sea kayak', 6)]

    #Create a SelectMultiple widget (Multiselect Dropdown List):
    vessels = Dropdown(options = vessels,
                              value = 5,
                              rows = 6,
                              description = 'Vessel type',
                              disabled = False)


    #Create function that updates the input parameters to your other
    #python functions, based on the user's input (selections):
    
    def update_func(vessel_selection):
        
        
 
        list_selection=[]

        #for value in vessel_selection:
        list_selection.append(vessel_selection)
            

        
        
    #Create function that contains a box of widgets:

    interact_c = interact(update_func, vessel_selection=vessels)
    
    
    return vessels
    
    

    
    #Format widget:
    interact_c.widget.children[0].layout.width = '430px' #adjusts the width of the dropdown box
    
    



In [ ]:
# Description: Estimates the leeway drift (in km)  occurred during time
#             interval Dt,  based on the USCG parameters.
#             Adds to it drift due to local currents. Leeways can
#             be estimated for different types of craft.  
#
# Input:   Ws_ms = Wind speed  at 10 meters  in m/s.
#                 Vector with two elements.Ws_ms(1)=zonal, Ws_ms(2)=merid.
#         Cs_ms = 2 element current vector in m/s. Cs_ms(1)=zonal, Cs_ms(2)=merid.
#         Dt    = time interval, in seconds
#         Crf   = Type of craft (1=sanpan, 2=skiff, 3=sailboat,
#                                4=raft w/sail, 5=raft w/t sail
#                                 6=sea kayak
# Output:  T_drift(1) = zonal displacement in km
#         T_drift(2) = merid. displacement in km
####################################################################
#
#
#################### Leeway drift EQUATIONS ##############################
#                    from the USCG handbook                    
#
# Eq1:  Leeway speed (knots) = [Slope * Wind Speed (knots)] + ...
#  Y_intercept (knots), for wind speeds greater than six knots 
#
#  LS_K=(Sl*Ws_K)+Yt 
#
# Eq2:  Leeway speed (knots) = [Slope + Y_intercept/6] * Wind ...
#    Speed (knots), for wind speeds less than six knots.
#
#  
#  LS_K=(Sl+(Yt/6))*Ws_K
#
#####################################################################
#
##############  LEEWAY TAXONOMY AND PARAMETERS #################
#
#  1) Commercial Fishing Vessels.  Include vessels from 45 to 100
#     feet long designed for fishing or shell fishing in coastal and
#     ocean waters.All have some form of deckhouse and an open area 
#     from which nets can lines are worked. 
#     USE - Sampan
#     Sl=0.04; Yt=0; Da=48;    Error, knosts=25.
#
#  2) Skiffs.  Open boats less than 20 ft long.
#     USE - V-hull standard 
#     Sl=0.03; Yt=0.08; Da=15; Error, knosts=0.10
#
#  3) Mono-hull Sailing Vessel.  It is assumed that all targets 
#     in this category are adrift; therefore sails are down or 
#     missing and the crew is unable to maneuver the vessel at all.
#     USE -  Fin Keel.
#     Sl=0.04; Yt=0; Da=48; Error, knosts=0.25
#
#  4) Rustic raft with sail. 
#     USE -  sail
#     Sl=0.08; Yt=-0.17; Da=33; Error, knosts=0.15
#
#  5) Rustic raft without sail
#     USE - no sail  
#      Sl=0.015; Yt=0.17; Da=17; Error, knosts=0.05
#  
#  6) Sea kayak
#     USE 1 person
#      Sl=0.011; Yt=-0.24; Da=15; Error, knosts=0.10
#  
#  7) Levison et al. parameters. Wind component of vessel movement
#     is in the same direction of wind with speeds given 
#     by their Table 2, page 19 of the book. 
#
######################################################################

def USCG_Drift1(Ws_ms, Cs_ms, Dt, Crft):
    ############ Choice of craft ###################

    if Crft == 1:
        # sampan
        Sl = 0.04
        Yt = 0.00
        Da = 48
    elif Crft == 2:
        # skiff
        Sl = 0.03
        Yt = 0.08
        Da = 15
    elif Crft == 3:
        # sailboat
        Sl = 0.03
        Yt = 0.00
        Da = 48
    elif Crft == 4:
        # sail raft
        Sl = 0.08
        Yt = -0.17
        Da = 33
    elif Crft == 5:
        # no sail raft
        Sl = 0.015
        Yt = 0.17
        Da = 17
    elif Crft == 6:
        # sea kayak
        Sl = 0.011
        Yt = 0.24
        Da = 15
    else:
        raise RuntimeError('Unknown craft type')

    ##### Convertions 1 ####################
    ####  knots=m/s*1.94

    Ws_k = Ws_ms * 1.94  # Ws_ms to knots

    ############# The current drift

    Dx_Uc = Cs_ms[0] * Dt  # current zonal drift
    Dy_Vc = Cs_ms[1] * Dt  # current merid. drift

    Ls_k = np.zeros((2,))
    Ld = np.zeros((2,))
    T_drft = np.zeros((2,))
    if Crft != 7:

        Da_rad = (2 * np.pi * Da) / 360  # Da to radians

        ################# total leeway speed (Ls_k) depending on Ws_k

        if abs(Ws_k[0]) > 6:
            Ls_k[0] = (Sl * Ws_k[0]) + Yt
        else:
            Ls_k[1] = (Sl + (Yt / 6)) * Ws_k[0]

        if abs(Ws_k[1]) > 6:
            Ls_k[1] = (Sl * Ws_k[1]) + Yt
        else:
            Ls_k[1] = (Sl + (Yt / 6)) * Ws_k[1]

        ##### Convertions 2 ###################
        ####  m/s=knots*0.51

        Ls_ms = Ls_k * 0.51

        #### use speed to calculate linear displacement Ld
        Ld[0] = Ls_ms[0] * Dt  # disp.due to U wind drift
        Ld[1] = Ls_ms[1] * Dt  # disp.due to V wind drift

        ### the deflections due to Da half right
        ### and  half left of the wind
        coin = np.random.rand()
        if coin > .5:
            flp = 1
        else:
            flp = -1

        Dx_Uw = Ld[0] * np.cos(Da_rad)  # zonal wind zonal drift
        Dy_Uw = Ld[0] * np.sin(Da_rad * flp)  # zonal wind merid. drift

        Dx_Vw = Ld[1] * np.sin(Da_rad * -flp)  # merid. wind zonal drift
        Dy_Vw = Ld[1] * np.cos(Da_rad)  # merid. wind merid. drift

        #########################################################
        ######### The total drift (wind +current) in km ##########

        T_drft[0] = (Dx_Uw + Dx_Vw + Dx_Uc) / 1e+3  # zonal
        T_drft[1] = (Dy_Uw + Dy_Vw + Dy_Vc) / 1e+3  # merid.

    ########### total leeway for the Levison method.
    if Crft == 7:

        aWs_k = abs(Ws_k)
        sz = np.sign(Ws_k[0])
        sm = np.sign(Ws_k[1])
        # zonal component
        if aWs_k[0] < 1:
            Ls_k[0] = 0
        elif 1 <= aWs_k[0] <= 3:
            Ls_k[0] = 0.5 * sz
        elif 3 < aWs_k[0] <= 6:
            Ls_k[0] = 1 * sz
        elif 6 < aWs_k[0] <= 10:
            Ls_k[0] = 2 * sz
        elif 10 < aWs_k[0] <= 16:
            Ls_k[0] = 3 * sz
        elif 16 < aWs_k[0] <= 21:
            Ls_k[0] = 4.5 * sz
        elif 21 < aWs_k[0] <= 27:
            Ls_k[0] = 6 * sz
        elif 27 < aWs_k[0] <= 33:
            Ls_k[0] = 7 * sz
        elif 33 < aWs_k[0] <= 40:
            Ls_k[0] = 6 * sz
        elif aWs_k[0] > 40:
            Ls_k[0] = 4.5 * sz

        # meridional component
        if aWs_k[1] < 1:
            Ls_k[1] = 0
        elif 1 <= aWs_k[1] <= 3:
            Ls_k[1] = 0.5 * sm
        elif 3 < aWs_k[1] <= 6:
            Ls_k[1] = 1 * sm
        elif 6 < aWs_k[1] <= 10:
            Ls_k[1] = 2 * sm
        elif 10 < aWs_k[1] <= 16:
            Ls_k[1] = 3 * sm
        elif 16 < aWs_k[1] <= 21:
            Ls_k[1] = 4.5 * sm
        elif 21 < aWs_k[1] <= 27:
            Ls_k[1] = 6 * sm
        elif 27 < aWs_k[1] <= 33:
            Ls_k[1] = 7 * sm
        elif 33 < aWs_k[1] <= 40:
            Ls_k[1] = 6 * sm
        elif aWs_k[1] > 40:
            Ls_k[1] = 4.5 * sm

        # converting from knots to m/s
        Ls_ms = Ls_k * 0.51

        #### use speed to calculate linear displacement Ld
        Ld[0] = Ls_ms[0] * Dt  # disp.due to U wind drift
        Ld[1] = Ls_ms[1] * Dt  # disp.due to V wind drift

        T_drft[0] = (Ld[0] + Dx_Uc) / 1e+3  # zonal
        T_drft[1] = (Ld[1] + Dy_Vc) / 1e+3  # merid.

    return T_drft

In [5]:
def draw_studyarea():
    import pickle

    from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl

    watercolor = basemap_to_tiles(basemaps.OpenStreetMap.France)

    m = Map(layers=(watercolor, ), center=(50, 354), zoom=5)

    draw_control = DrawControl(rectangle={'shapeOptions': {'color': '#0000FF'}})
    """
    draw_control.polyline =  {
        "shapeOptions": {
            "color": "#6bc2e5",
            "weight": 8,
            "opacity": 1.0
        }
    }
    draw_control.polygon = {
        "shapeOptions": {
            "fillColor": "#6be5c3",
            "color": "#6be5c3",
            "fillOpacity": 1.0
        },
        "drawError": {
            "color": "#dd253b",
            "message": "Oups!"
        },
        "allowIntersection": False
    }
    draw_control.circle = {
        "shapeOptions": {
            "fillColor": "#efed69",
            "color": "#efed69",
            "fillOpacity": 1.0
        }
    }
    draw_control.rectangle = {
        "shapeOptions": {
            "fillColor": "#fca45d",
            "color": "#fca45d",
            "fillOpacity": 1.0
        }
    }

    """

    #add what happens when draw something
    def handle_draw(self, action, geo_json):
        #print(action)
        print(geo_json)
        #print(geo_json['geometry']['coordinates'])
        
        list_coordinates=geo_json['geometry']['coordinates']
        
        if geo_json['geometry']['type']== 'LineString':
            save_dep_points = "dep_points.txt"

            with open(save_dep_points, "wb") as f:
                pickle.dump(list_coordinates[0], f)

        if geo_json['geometry']['type']== 'Polygon':
            save_area = "study_area.txt"

            with open(save_area, "wb") as f:
                pickle.dump(list_coordinates[0], f)


    draw_control.on_draw(handle_draw)
    m.add_control(draw_control)

    m
    return m
